## CSP Type:
1, CSSP: Common Spatial Spectral Pattern

2, SWCSP: Spectrally Weighted Common Spatial Patterns

3, ISSPL: Iterative Spatio Spectral Patterns Learning

4, FBCSP: Filter Bank Common Spatial Patterns

5, SCSSP: Seperable Common Spatio-spectral patterns
